In [1]:
#-*- coding:utf-8 -*-
from commonTool import *
import glob


running on Windows


In [ ]:
#################################################
# EVALUATE-PREPARE.xml 실행되어야 함.

In [2]:
# 기준일 로딩

bf = open(resourceDir + 'BASIS-DAYS.txt', 'r', encoding='utf-8')

# 0.YYYYMM, 1.FIRST_DAY, 2.SECOND_DAY, 3.PREV_DAY, 4.LAST_DAY, 5.NEXT_DAY
lineText = bf.readline() # 제목

basisDays = []
quaterBasisDays = []

while True:
    lineText = bf.readline()
    if not lineText:
        break
        
    lineText = lineText.strip()
    if len(lineText) <= 0:
        continue

    dayData = lineText.split('\t')
    
    if dayData[0][0:4] == '2012' or dayData[0] == '201301': continue # 데이터 부족
        
    basisDays.append([dayData[0], dayData[3], dayData[4]])
    
    if dayData[0] > '202001':
        quaterBasisDays.append([dayData[0], dayData[3], dayData[4]])

bf.close()


In [4]:
# 로직 적용한 결과 정리

def makeAnalysisReport(resultFolder, initialCash, period, feeRate = 0.015, withProcess = True):
    statResult = [f for f in glob.glob(resultFolder + '*-STAT.txt')]
    
    if len(statResult) == 0:
        print('no result file')
        return
    
    statResult.sort()
    
    m = 0
    y = 0
    cash = initialCash
    maxV = initialCash
    maxMdd = 0
    cagr = 0
    
    if withProcess:
        print('\t'.join(['Year', '매수일', '매도일', '손익', '투자금', '수익률', 'CAGR', 'MDD']))
    
    for statPath in statResult:
        m += period
        
        sf = open(statPath, 'r', encoding='utf-8')
        lineText = sf.readline() # 제목: --> 매수일, 매도일, R_SUM
        lineText = sf.readline() # 결과
        
        rec = lineText.split('\t')
        profitLoss = int(rec[2])

        newCash = round((cash + profitLoss) * (1.0 - feeRate))

        maxV = max(maxV, newCash)
        if newCash < cash:
            mdd = (newCash - maxV) / maxV
            maxMdd = min(mdd, maxMdd)

        y = m / 12
        cagr = pow(newCash / initialCash, 1 / y) - 1
        cagr = round(cagr * 100, 2)
        
        if withProcess:
            print('\t'.join([str(y), rec[0], rec[1],
                format(profitLoss, ',d').rjust(12), format(newCash, ',d').rjust(15),
                str(round(profitLoss / cash * 100, 2)).rjust(9), str(cagr).rjust(9), str(round(maxMdd * -100, 2))]))

        sf.close()
        cash = newCash

    if withProcess: print('')
    print('Year:', y, '  Invest:', format(initialCash, ',d'), '  Cash:', format(cash, ',d'),
        '  CAGR:', cagr, '%', '  Max MDD:', round(maxMdd * -100, 2), '%\n\n')


In [7]:
testResult = [
    '4PXX-25-10T-10-M2',
    '5PXR-25-10T-10-M2',
    # '4PXX-25-10T-15-M2',
    # '4PXX-25-10T-20-M2',
    'END' # 콤마 편하게 하려고 넣었음
]

for key in testResult:
    if key == 'END': break
        
    print('#', key)
    makeAnalysisReport(outputRawPath + 'logicResult' + os.path.sep + key + os.path.sep, 10000000,
                       (3 if key.endswith('-P3') else 6), 0.015, True)


# 4PXX-25-10T-10-M2
Year: 8.5   Invest: 10,000,000   Cash: 131,481,824   CAGR: 35.4 %   Max MDD: 6.49 %


# 5PXR-25-10T-10-M2
Year: 8.5   Invest: 10,000,000   Cash: 111,727,486   CAGR: 32.84 %   Max MDD: 3.91 %




In [ ]:
testResult = [
    '4PXX-25-10T-10-M2',
    '4PXX-25-10T-10-M3',
    '4PXX-25-10T-10-M4',
    '4PXX-25-10T-10-M5',
    '4PXX-25-10T-10-M6',
    'END' # 콤마 편하게 하려고 넣었음
]

for key in testResult:
    if key == 'END': break
        
    print('#', key)
    makeAnalysisReport(outputRawPath + 'logicResult' + os.path.sep + key + os.path.sep, 10000000, 6, 0.015, True)


In [10]:
testResult = [
    # '4PXX-25-10T-20-M2',
    'Q-4PXX-25-10T-10-M2',
    'Q-4PXX-25-10T-10-M3',
    'Q-4PXX-25-10T-10-M4',
    'END' # 콤마 편하게 하려고 넣었음
]

for key in testResult:
    if key == 'END': break
        
    print('#', key)
    makeAnalysisReport(outputRawPath + 'logicResult' + os.path.sep + key + os.path.sep, 10000000, 3, 0.015, False)


# Q-4PXX-25-10T-10-M2
Year: 1.5   Invest: 10,000,000   Cash: 27,212,360   CAGR: 94.91 %   Max MDD: 0 %


# Q-4PXX-25-10T-10-M3
Year: 1.5   Invest: 10,000,000   Cash: 29,400,496   CAGR: 105.23 %   Max MDD: 1.5 %


# Q-4PXX-25-10T-10-M4
Year: 1.25   Invest: 10,000,000   Cash: 24,515,198   CAGR: 104.9 %   Max MDD: 0 %




In [8]:
makeAnalysisReport(outputRawPath + 'logicResult' + os.path.sep + '4PXX-25-10T-10-M2' + os.path.sep, 10000000, 6, 0.015, True)

Year	매수일	매도일	손익	투자금	수익률	CAGR
0.5	20130228	20130830	   1,804,682	     11,627,612	    18.05	     35.2
1.0	20130830	20140228	     943,980	     12,383,018	     8.12	    23.83
1.5	20140228	20140829	   3,306,281	     15,453,960	     26.7	    33.67
2.0	20140829	20150227	    -782,621	     14,451,269	    -5.06	    20.21
2.5	20150227	20150831	   1,184,610	     15,401,341	      8.2	    18.86
3.0	20150831	20160229	    -296,476	     14,878,292	    -1.93	    14.16
3.5	20160229	20160831	   4,609,739	     19,195,711	    30.98	    20.48
4.0	20160831	20170228	   8,982,607	     27,755,643	    46.79	    29.07
4.5	20170228	20170831	   4,574,559	     31,845,249	    16.48	    29.36
5.0	20170831	20180228	   7,914,065	     39,162,924	    24.85	    31.39
5.5	20180228	20180831	   9,515,329	     47,948,079	     24.3	    32.98
6.0	20180831	20190228	    -812,796	     46,428,254	     -1.7	    29.16
6.5	20190228	20190830	   7,018,282	     52,644,838	    15.12	    29.12
7.0	20190830	20200228	  -1,428,193	     50,448,3

In [3]:
# 년간 재무 데이터를 이용한 분석

analId = '5PXR'  # 로직명: 4PXX, MAGIC, %PXR
inital_cash = 10000000  # 초기 투자금
period = 6  # 리밸런싱 기간 (월)
buyCount = 10  # 매수 종목 개수
totalAmount = 25 # 시가총액 하위 퍼센트 조건
volumeLimit = 10000 # 거래량 조건

startMonth = 2

analTitle = analId + '-' + str(totalAmount) + '-' + str(round(volumeLimit / 1000)) + 'T-' + str(buyCount) \
    + '-M' + str(startMonth) + ('-P' + str(period) if period != 6 else '')

print('running', analTitle)


feeRate = 0.015 # 1.5% 수수료

# 로직 스크립트
scriptPathName = crawlegoScriptPath + 'ANAL-' + analId + '.xml'

# 결과 저장 위치
resultFolder = outputRawPath + 'logicResult' + os.path.sep + analTitle + os.path.sep
# [os.remove(f) for f in glob.glob(resultFolder + '*.txt')]
mkdir(resultFolder)

# 계산 시작
m = 0
cash = inital_cash

for i in range(startMonth - 2, len(basisDays), period):
    basis = basisDays[i]
    sellIdx = min(i + period, len(basisDays) - 1)
    
    parameter = {
        'BASIS_DAY': basis[1],
        'BUY_DAY': basis[2],
        'SELL_DAY': basisDays[sellIdx][2],
        'CASH': str(cash),
        'TOP_N': str(buyCount),
        'PERIOD': 'ANNUAL',
        'IN_PATH': outputRawPath + 'index' + os.path.sep + (os.path.sep if runServerType == 1 else ''),
        'OUT_PATH': resultFolder + (os.path.sep if runServerType == 1 else ''),
        'TOTAL_AMOUNT': str(totalAmount),
        'VOL_LIMIT': str(volumeLimit),
        'DO_SERVER': '13.124.29.70'
    }

    m += period
    retCode = runDashScript(scriptPathName, parameter, False)
    
    print(m / 12, basis[0], 'ret code', retCode)
    
    if retCode != 0:
        continue

    # 결과 정리
    statPath = resultFolder + basis[1] + '-STAT.txt'
    sf = open(statPath, 'r', encoding='utf-8')
    
    lineText = sf.readline() # 제목: 매수일, 매도일, R_SUM
    lineText = sf.readline() # 결과
    
    rec = lineText.split('\t')
    profitLoss = int(rec[2])
    sf.close()
    
    cash = round((cash + profitLoss) * (1.0 - feeRate))
    # end of for

makeAnalysisReport(resultFolder, inital_cash, period, feeRate, True)
# end of script

running 5PXR-25-10T-10-M2
0.5 201302 ret code 0
1.0 201308 ret code 0
1.5 201402 ret code 0
2.0 201408 ret code 0
2.5 201502 ret code 0
3.0 201508 ret code 0
3.5 201602 ret code 0
4.0 201608 ret code 0
4.5 201702 ret code 0
5.0 201708 ret code 0
5.5 201802 ret code 0
6.0 201808 ret code 0
6.5 201902 ret code 0
7.0 201908 ret code 0
7.5 202002 ret code 0
8.0 202008 ret code 0
8.5 202102 ret code 0


NameError: name 'makeAnalysisReport' is not defined

In [ ]:
# 분기 재무 데이터를 이용한 분석

analId = '4PXX'  # 로직명
inital_cash = 10000000  # 초기 투자금
period = 3  # 리밸런싱 기간 (월)
buyCount = 10  # 매수 종목 개수
totalAmount = 25 # 시가총액 하위 퍼센트 조건
volumeLimit = 10000 # 거래량 조건

startMonth = 2

analTitle = 'Q-' + analId + '-' + str(totalAmount) + '-' + str(round(volumeLimit / 1000)) + 'T-' + str(buyCount) + '-M' + str(startMonth)

print('running', analTitle)


feeRate = 0.015 # 1.5% 수수료

# 로직 스크립트
scriptPathName = crawlegoScriptPath + 'ANAL-' + analId + '.xml'

# 결과 저장 위치
resultFolder = outputRawPath + 'logicResult' + os.path.sep + analTitle + os.path.sep
# [os.remove(f) for f in glob.glob(resultFolder + '*.txt')]
mkdir(resultFolder)

# 계산 시작
m = 0
cash = inital_cash

for i in range(startMonth - 2, len(quaterBasisDays), period):
    basis = quaterBasisDays[i]
    sellIdx = min(i + period, len(quaterBasisDays) - 1)
    
    parameter = {
        'BASIS_DAY': basis[1],
        'BUY_DAY': basis[2],
        'SELL_DAY': quaterBasisDays[sellIdx][2],
        'CASH': str(cash),
        'TOP_N': str(buyCount),
        'PERIOD': 'QUARTER',
        'IN_PATH': outputRawPath + 'index' + os.path.sep + (os.path.sep if runServerType == 1 else ''),
        'OUT_PATH': resultFolder + (os.path.sep if runServerType == 1 else ''),
        'TOTAL_AMOUNT': str(totalAmount),
        'VOL_LIMIT': str(volumeLimit),
        'DO_SERVER': '13.124.29.70'
    }

    m += period
    retCode = runDashScript(scriptPathName, parameter, False)
    
    print(m / 12, basis[0], 'ret code', retCode)
    
    if retCode != 0:
        continue

    # 결과 정리
    statPath = resultFolder + basis[1] + '-STAT.txt'
    sf = open(statPath, 'r', encoding='utf-8')
    
    lineText = sf.readline() # 제목: 매수일, 매도일, R_SUM
    lineText = sf.readline() # 결과
    
    rec = lineText.split('\t')
    profitLoss = int(rec[2])
    sf.close()
    
    cash = round((cash + profitLoss) * (1.0 - feeRate))
    # end of for

makeAnalysisReport(resultFolder, inital_cash, period, feeRate, True)
# end of script

In [5]:
resultFolder = outputRawPath + 'logicResult' + os.path.sep + '4PXX-25-10T-10-M2' + os.path.sep
saveFolder = temporaryPath + 'analMM' + os.path.sep

scriptPathName = crawlegoScriptPath + 'CALC-MOMENTUM.xml'

mmCount = 3
analResult = [f for f in glob.glob(resultFolder + '*-RESULT.txt')]

for selFile in analResult:
    p = selFile.rfind(os.path.sep)
    basisDay = selFile[p+1:p+9]
    
    parameter = {
        'BASIS_DAY': basisDay,
        'IN_PATH': resultFolder + (os.path.sep if runServerType == 1 else ''),
        'OUT_PATH': saveFolder + (os.path.sep if runServerType == 1 else ''),
        'MM_COUNT': str(mmCount),
        'DO_SERVER': '13.124.29.70'
    }

    retCode = runDashScript(scriptPathName, parameter, False)
    print(basisDay, 'returns', retCode)
    

20130227 returns 0


In [19]:
resultFolder = outputRawPath + 'logicResult' + os.path.sep + '4PXX-25-10T-10-M2' + os.path.sep
saveFolder = temporaryPath + 'analMM' + os.path.sep

scriptPathName = crawlegoScriptPath + 'CALC-AVGINDEX.xml'

mmCount = 3
analResult = [f for f in glob.glob(resultFolder + '*-RESULT.txt')]

for selFile in analResult:
    p = selFile.rfind(os.path.sep)
    basisDay = selFile[p+1:p+9]
    
    parameter = {
        'BASIS_DAY': basisDay,
        'IN_PATH': resultFolder + (os.path.sep if runServerType == 1 else ''),
        'OUT_PATH': saveFolder + (os.path.sep if runServerType == 1 else ''),
        'DO_SERVER': '13.124.29.70'
    }

    retCode = runDashScript(scriptPathName, parameter, False)
    print(basisDay, 'returns', retCode)
    

20130227 returns 0
20130829 returns 0
20140227 returns 0
20140828 returns 0
20150226 returns 0
20150828 returns 0
20160226 returns 0
20160830 returns 0
20170227 returns 0
20170830 returns 0
20180227 returns 0
20180830 returns 0
20190227 returns 0
20190829 returns 0
20200227 returns 0
20200828 returns 0
20210225 returns 0


In [7]:
DD = [
    ['20130227', '20130228', '20130731'],
    ['20130829', '20130930', '20131129'],
    ['20140227', '20140228', '20140829'],
    ['20140828', '20140828', '20140828'],
    ['20150226', '20150227', '20150630'],
    ['20150828', '20150828', '20150828'],
    ['20160226', '20160229', '20160831'],
    ['20160830', '20160831', '20170228'],
    ['20170227', '20170331', '20170831'],
    ['20170830', '20171130', '20180228'],
    ['20180227', '20180228', '20180731'],
    ['20180830', '20180830', '20180830'],
    ['20190227', '20190228', '20190830'],
    ['20190829', '20191031', '20200228'],
    ['20200227', '20200228', '20200831'],
    ['20200828', '20201030', '20210226'],
    ['20210225', '20210226', '20210621']
]

resultFolder = outputRawPath + 'logicResult' + os.path.sep + '4PXX-25-10T-10-M2' + os.path.sep
saveFolder = 'D:\work\jupyter\krx\intermediate\profit' + os.path.sep

scriptPathName = crawlegoScriptPath + 'CALC-PROFIT.xml'

cash = 10000000
for day in DD:
    parameter = {
        'BASIS_DAY': day[0],
        'BUY_DAY': day[1],
        'SELL_DAY': day[2],
        'CASH': str(cash),
        'IN_PATH': resultFolder + (os.path.sep if runServerType == 1 else ''),
        'OUT_PATH': saveFolder + (os.path.sep if runServerType == 1 else ''),
        'DO_SERVER': '13.124.29.70'
    }

    retCode = runDashScript(scriptPathName, parameter, False)
    print(day[0], 'returns', retCode)

    sf = open(saveFolder + day[0] + '-STAT.txt', 'r', encoding='utf-8')
    lineText = sf.readline() # 제목: --> 매수일, 매도일, R_SUM
    lineText = sf.readline() # 결과

    rec = lineText.split('\t')
    profitLoss = int(rec[2])
    
    sf.close()

    cash += profitLoss
    print(day[0], profitLoss, cash)


20130227 returns 0
20130227 1676677 11676677
20130829 returns 0
20130829 -248092 11428585
20140227 returns 0
20140227 3065977 14494562
20140828 returns 0
20140828 -29023 14465539
20150226 returns 0
20150226 -812285 13653254
20150828 returns 0
20150828 97490 13750744
20160226 returns 0
20160226 4556302 18307046
20160830 returns 0
20160830 8537447 26844493
20170227 returns 0
20170227 -5007457 21837036
20170830 returns 0
20170830 1847390 23684426
20180227 returns 0
20180227 3787583 27472009
20180830 returns 0
20180830 16080 27488089
20190227 returns 0
20190227 3510661 30998750
20190829 returns 0
20190829 581888 31580638
20200227 returns 0
20200227 1233924 32814562
20200828 returns 0
20200828 14787355 47601917
20210225 returns 0
20210225 22997330 70599247
